<a href="https://colab.research.google.com/github/dmswl0707/light_weighted_CNN_for_FER/blob/main/FER2013_train_test_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
from torch import optim
import copy
import time
import torch
from torch.utils import data
from torch import nn

from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/My Drive/Colab Notebooks/FER2013/train/'
test_dir = '/content/drive/My Drive/Colab Notebooks/FER2013/test/'



In [ ]:
### Data augmentation

transforms = transforms.Compose([transforms.Resize((48,48)),
                                 transforms.Grayscale(),
                                 transforms.RandomRotation(degrees=30),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor()
                                 ])

In [ ]:
train_on_gpu = torch.cuda.is_available()
print(train_on_gpu)

In [ ]:
### 데이터 불러오기

df_train = ImageFolder(root= train_dir, transform = transforms)
df_test = ImageFolder(root = test_dir, transform = transforms)


In [ ]:
# test 갯수와 카테고리 확인

categories = list(df_test.class_to_idx.keys())
print(categories)
num_classes = len(categories)

print(len(df_test))
#print(len(num_classes))


In [ ]:
#### train / test 데이터 데이터 구성확인

def get_count(path,set_):
    
    dict_ = {}
    
    for exp in os.listdir(path):
        dir_ = path + exp
        dict_[exp] = len(os.listdir(dir_))
        
    df = pd.DataFrame(dict_,index=[set_])
    return df
    

train_count = get_count(train_dir,'train')
test_count = get_count(test_dir,'test')

print(train_count)
print(test_count)

In [ ]:
#train, test set의 class 분포 확인
'''
emotions = {0 : 'angry', 1: 'disgust', 2 : 'fear', 3 : 'happy', 4 : 'sad', 5 : 'surprise', 6: 'neutral'}

em_count = df_train['emotion'].value_counts().plot(kind='bar')
plt.xlabel("total", labelpad=14)
plt.ylabel("emotions", labelpad=14)
plt.title("value_counts by emotions_[train]", y=1.02);
'''

In [ ]:
### train /val split

val_size = 0.25 # train set의 0.2 사이즈 만큼

num_data = len(df_train)
indices = list(range(num_data))
np.random.shuffle(indices)


val_split = int(np.floor(val_size * num_data))
val_idx, train_idx = indices[:val_split], indices[val_split:]


print(len(train_idx), len(val_idx))

In [ ]:

###데이터 로더

dataset_sizes = {
    'train' : len(train_idx),
    'test' : len(df_test),
    'val' : len(val_idx)
}

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)
test_sampler = SubsetRandomSampler(df_test)

batch_size=16

loaders = {
    'train': torch.utils.data.DataLoader(df_train, batch_size=16, sampler=train_sampler),
    'val': torch.utils.data.DataLoader(df_train, batch_size=16, sampler=val_sampler),
}
test_loader = torch.utils.data.DataLoader(df_test, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
## 데이터 확인하기

def imshow(img):
    img = (img+1)/2    
    img = img.squeeze()
    np_img = np.asarray(img)
    plt.imshow(np_img)


dataiter = iter(loaders['val'])
images, labels = dataiter.next()
print(images.shape,labels.shape)
images = images.numpy() 
fig = plt.figure(figsize=(25, 10))

for idx in np.arange(8):
    ax = fig.add_subplot(2, 4, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(categories[int(labels[idx])],fontsize=20,color='white')

In [ ]:
import torch.nn as nn
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F


class SeparableConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, dilation, groups=in_channels,
                                   bias=bias)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


class ResidualBlock(nn.Module):

    def __init__(self, in_channeld, out_channels):
        super(ResidualBlock, self).__init__()

        self.residual_conv = nn.Conv2d(in_channels=in_channeld, out_channels=out_channels, kernel_size=1, stride=2,
                                       bias=False)
        self.residual_bn = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)

        self.sepConv1 = SeparableConv2d(in_channels=in_channeld, out_channels=out_channels, kernel_size=3, bias=False,
                                        padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.elu = nn.ELU()

        self.sepConv2 = SeparableConv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, bias=False,
                                        padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.maxp = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        res = self.residual_conv(x)
        res = self.residual_bn(res)
        x = self.sepConv1(x)
        x = self.bn1(x)
        x = self.elu(x)
        x = self.sepConv2(x)
        x = self.bn2(x)
        x = self.maxp(x)
        return res + x


class Block(nn.Module):

    def __init__(self, in_channeld, out_channels):
        super(Block, self).__init__()

        self.sep1 = SeparableConv2d(in_channels=in_channeld, out_channels=out_channels, kernel_size=3, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.elu1 = nn.ELU()
        self.sep2 = SeparableConv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.elu2 = nn.ELU()
        self.maxp = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.sep1(x)
        x = self.bn1(x)
        x = self.elu1(x)

        x = self.sep2(x)
        x = self.bn2(x)
        x = self.elu2(x)
        x = self.maxp(x)

        return x


class SELayer(nn.Module):
    def __init__(self, in_channeld, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channeld, in_channeld // reduction, bias=False),
            nn.ReLU6(inplace=True),
            nn.Linear(in_channeld // reduction, in_channeld, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channeld, out_channels, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=16):
        super(SEBasicBlock, self).__init__()
        self.Conv1 = nn.Conv2d(in_channeld, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(in_channeld)
        self.relu = nn.ReLU6(inplace=True)

        self.Conv2 = nn.Conv2d(in_channeld, out_channels, 1)
        self.bn2 = nn.BatchNorm2d(in_channeld)
        self.se = SELayer(in_channeld, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.Conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.Conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class Model(nn.Module):

    def __init__(self, num_classes):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, dilation=2, bias=False)
        self.bn1 = nn.BatchNorm2d(8, affine=True, momentum=0.99, eps=1e-3)
        self.relu1 = nn.ELU()
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, dilation=2, bias=False)
        self.bn2 = nn.BatchNorm2d(8, momentum=0.99, eps=1e-3)
        self.relu2 = nn.ELU()


        self.module1 = ResidualBlock(in_channeld=8, out_channels=16)
        self.module2 = SEBasicBlock(16, 16, stride=1, downsample=None, groups=1,
                                    base_width=64, dilation=1, norm_layer=None, reduction=16)
        self.module3 = Block(in_channeld=16, out_channels=32)
        self.module4 = ResidualBlock(in_channeld=32, out_channels=64)
        self.module5 = SEBasicBlock(64, 64, stride=1, downsample=None, groups=1,
                                    base_width=64, dilation=1, norm_layer=None, reduction=16)
        self.module6 = Block(in_channeld=64, out_channels=128)


        self.last_conv = nn.Conv2d(in_channels=128, out_channels=num_classes, kernel_size=3, padding=1, stride=2)
        self.avgp = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, input):
        x = input
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.module1(x)
        x = self.module2(x)
        x = self.module3(x)
        x = self.module4(x)
        x = self.module5(x)
        x = self.module6(x)
        x = self.last_conv(x)
        x = self.avgp(x)
        x = x.view((x.shape[0], -1))
        return x

In [ ]:
def graph_loss(train_loss, val_loss):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax.plot(train_loss)
    ax.plot(val_loss)
    ax.legend(['Train Loss','val Loss'])


In [ ]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
patience = 20

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    train_losses, val_losses = [], []
    avg_train_losses, avg_val_losses = [], []

    since = time.time()

    early_stopping = EarlyStopping(patience = patience, verbose = True)

    for epoch in range(1, num_epochs +1) :
            #print('Epoch {}/{}'.format(epoch + 1, num_epochs))
            #print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
            # Each epoch has a training and validation phase
        
                model.train()  # Set model to training mode

                #running_loss = 0.0
                #corrects = 0

                # Iterate over data.
                for batch, (inputs, labels) in enumerate(loaders['train'], 1):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        #_, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                     # statistics
                    train_losses.append(loss.item())

                    #corrects += torch.sum(preds == labels.data)

                #scheduler.step()

                #epoch_loss = running_loss / dataset_sizes[phase]
                #epoch_acc = corrects.double() / dataset_sizes[phase]

                
                #train_loss.append(epoch_loss)
                #print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, train_losses, epoch_acc))


            if phase == 'val':
                model.eval()
                
                with torch.no_grad():
                    #val_loss = 0.0
                    #val_corrects = 0
                    
                    for batch, (inputs, labels) in enumerate(loaders['val']):
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                          
                        #forward
                        outputs = model(inputs)

                        #_, preds = torch.max(outputs, 1)
                        val_loss = criterion(outputs, labels)
                        val_losses.append(val_loss.item())
                        #val_corrects += torch.sum(preds == labels.data)

                    #val_acc = val_corrects.double() / dataset_sizes[phase]    
                    #print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, val_losses, val_acc))
                 

                    train_loss = np.average(train_losses)
                    val_loss = np.average(val_losses)
                    avg_train_losses.append(train_loss)
                    avg_val_losses.append(val_loss)

                    epoch_len = len(str(num_epochs))


                    print_msg = (f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] ' +
                                f'train_loss: {train_loss:.4f} ' +
                                f'valid_loss: {val_loss:.4f}')

                    print(print_msg)

            # deep copy the model
            #if phase == 'val' and epoch_acc > best_acc:
                #best_acc = epoch_acc
                    train_losses = []
                    val_losses = []

                    early_stopping(val_loss, model)
                    if early_stopping.early_stop:
                        print("Early stopping")
                        break

    model.load_state_dict(torch.load('checkpoint.pt'))

    return model_ft, avg_train_losses, avg_val_losses

                #best_model_wts = copy.deepcopy(model.state_dict())



    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))

    graph_loss(train_losses, val_losses)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model_ft = Model(num_classes=7)
model_ft = model_ft.to(device)
print(summary(model_ft,(1,50,50)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), momentum=0.9,lr=0.02, weight_decay=5e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.001)

In [ ]:
model, train_loss, val_loss = train_model(model_ft, criterion, optimizer, exp_lr_scheduler, num_epochs=100)

In [ ]:
# 훈련이 진행되는 과정에 따라 loss를 시각화
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(val_loss)+1),val_loss,label='Validation Loss')

# validation loss의 최저값 지점을 찾기
minposs = val_loss.index(min(val_loss))+1
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0, 0.5) # 일정한 scale
plt.xlim(0, len(train_loss)+1) # 일정한 scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches = 'tight')

In [ ]:
class_correct = list(0. for i in range(7))
class_total = list(0. for i in range(7))

correct = 0
total = 0

with torch.no_grad():
    for data in loaders['test']:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()  #예측과 실제 라벨 비교
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))            
            
for i in range(7):
    print('Accuracy of %5s : %2d %%' % (
        categories[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            fig = plt.figure(figsize=(25, 16))

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = fig.add_subplot(2, num_images//2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_labels[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model_ft = Model(num_classes=7)
model_ft = model_ft.to(device)
print(summary(model_ft,(1,44,44)))
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), momentum=0.9,lr=0.02, weight_decay=5e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)